In [1]:
import numpy as np
import pandas as pd
import networkx as nx


from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.base import clone

In [4]:
#preparing the data

X_df = pd.read_csv('../../data/presence_matrices/Campylobacter_coli_presence_absence.csv', index_col=0)
y_df= pd.read_csv('../../data/phenotypes/Campylobacter_coli_ciprofloxacin.csv', index_col=0)

for gene in X_df.columns:
    if X_df[gene].std() == 0:
        X_df.drop(gene, axis=1, inplace=True)

X_df = X_df.sort_index()
y_df = y_df.sort_index()

y_indices=list(y_df.index)
X_indices=list(X_df.index)

#intersection
intersection = [i for i in y_indices if i in X_indices]
y_df = y_df.loc[intersection]
X_df = X_df.loc[intersection]

X = X_df.values
y = y_df.values

In [3]:
#building an ensemble of 200 SVM with bootstrap and SGD+L1 regularization

n_models = 200
models = []
for i in range(n_models):
    X_boot, y_boot = resample(X, y, random_state=i)
    model = make_pipeline(StandardScaler(), SGDClassifier(loss='hinge', penalty='l1', alpha=0.0001, max_iter=1000, tol=1e-3))
    model.fit(X_boot, y_boot)
    models.append(model)
    

/home/raysas/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/raysas/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/raysas/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/raysas/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [ ]:
#get weights of SVM



In [4]:
weights = np.zeros((n_models, X.shape[1]))
for i, model in enumerate(models):
    weights[i] = model[1].coef_

In [8]:
#make weights  a df of cols = features and rows = models
weights_df = pd.DataFrame(weights, columns=X_df.columns)

corr_SVM_models= weights_df.corr()

In [10]:
corr_SVM_models

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,...,Cluster 8011,Cluster 8012,Cluster 8013,Cluster 8014,Cluster 8015,Cluster 8016,Cluster 8017,Cluster 8018,Cluster 8019,Cluster 8020
Cluster 0,1.000000,0.115557,-0.125107,0.067560,0.051637,0.010316,-0.145945,0.016273,0.022502,-0.266833,...,0.104166,-0.006691,0.044895,-0.003698,-0.008135,0.104166,0.104166,-0.001603,-0.059053,-0.054755
Cluster 1,0.115557,1.000000,-0.180199,0.100378,0.008447,0.014520,0.012898,-0.058000,-0.052309,0.024368,...,0.113897,0.112024,0.044513,-0.019307,-0.011691,0.113897,0.113897,0.048550,-0.080162,-0.019414
Cluster 2,-0.125107,-0.180199,1.000000,-0.074607,0.005335,-0.054430,0.164368,0.092582,0.124065,0.092908,...,-0.160654,0.088259,0.115118,-0.013316,0.035548,-0.160654,-0.160654,-0.002572,0.102078,-0.088876
Cluster 3,0.067560,0.100378,-0.074607,1.000000,0.019621,-0.019730,0.059200,-0.121388,-0.022590,0.131859,...,0.050428,0.052994,-0.056994,-0.053122,-0.058330,0.050428,0.050428,-0.087125,0.009738,-0.077503
Cluster 4,0.051637,0.008447,0.005335,0.019621,1.000000,-0.085495,-0.054393,-0.181487,0.239926,0.120407,...,-0.062503,0.066108,0.095512,-0.007106,0.058009,-0.062503,-0.062503,0.030738,0.034670,-0.129304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cluster 8016,0.104166,0.113897,-0.160654,0.050428,-0.062503,-0.044846,-0.087017,-0.041950,-0.090350,-0.083812,...,1.000000,0.031163,0.073510,-0.003404,0.031794,1.000000,1.000000,0.013432,-0.037493,-0.100721
Cluster 8017,0.104166,0.113897,-0.160654,0.050428,-0.062503,-0.044846,-0.087017,-0.041950,-0.090350,-0.083812,...,1.000000,0.031163,0.073510,-0.003404,0.031794,1.000000,1.000000,0.013432,-0.037493,-0.100721
Cluster 8018,-0.001603,0.048550,-0.002572,-0.087125,0.030738,-0.023605,-0.062525,-0.173483,0.204637,-0.019114,...,0.013432,-0.100324,0.178160,-0.122037,-0.053748,0.013432,0.013432,1.000000,0.036313,-0.079634
Cluster 8019,-0.059053,-0.080162,0.102078,0.009738,0.034670,-0.029263,0.128972,-0.084996,-0.094300,-0.035112,...,-0.037493,-0.004457,-0.012147,-0.034018,-0.093475,-0.037493,-0.037493,0.036313,1.000000,0.008850


In [11]:
#sorted corr values:
corr_values = corr_SVM_models.abs().unstack().sort_values(ascending=False)
corr_values = corr_values[corr_values < 1]
corr_values

: 

In [ ]:
G=nx.Graph()

for i in range(X.shape[1]):
    for j in range(i+1, X.shape[1]):
        correlation = corr_SVM_models.iloc[i, j]
        if correlation > 0.6 or corr_SVM_models.iloc[j, i] < -0.6:
            G.add_edge(i, j, weight=correlation)